## Graph2SMILES performance evaluation

In [1]:
import random
from rdkit import Chem

predicted_product = []
ground_truth_product = []

with open('results/unseen_data/result.0', 'r') as output:
    output_lines = output.readlines()
    for line in output_lines:
        temp_line = line.split(',')[0].split('_')[0]
        predicted_product.append(temp_line)

with open('data/unseen_data/processed/tgt-test.txt') as target:
    target_lines = target.readlines()
    for line in target_lines:
        
        temp_line = line.replace(' ', '').strip()
        if 'END' in temp_line:
            temp_line = temp_line.replace('END', '')
            mol = Chem.MolFromSmiles(temp_line)
            temp_line = Chem.MolToSmiles(mol, isomericSmiles=True)
            temp_line = temp_line + "END"
        ground_truth_product.append(temp_line)

print(len(predicted_product))
print(len(ground_truth_product))

prediction_target_pair = list(zip(predicted_product, ground_truth_product))

printed = False
wrong_rxn_idx = []
for index, pair in enumerate(prediction_target_pair):
    if pair[0] != pair[1]:
        wrong_rxn_idx.append(index)
        if random.randint(0, 100) == 3 and printed == False:
            printed = True
            print(pair[0])
            print(' '.join(pair[1]))
            print(pair[1])
            print('\n')
        
print(wrong_rxn_idx)
print(len(wrong_rxn_idx))

4567
4567
CCOc1cc(C2C3=C(CC(CC)CC(=O)C2=C(C)NC(C)=C3C(=O)CC(CC)C3)C4)cc(Br)c1OCc1cccc(C([O-])(O)[NH2+]C2CCCCC2)c1
C C O c 1 c c ( C 2 C ( C # N ) = C ( C ) N C 3 = C 2 C ( = O ) C C ( C C ) C 3 ) c c ( B r ) c 1 O C c 1 c c c c ( C ( [ O - ] ) ( O ) [ N H 2 + ] C 2 C C C C C 2 ) c 1
CCOc1cc(C2C(C#N)=C(C)NC3=C2C(=O)CC(CC)C3)cc(Br)c1OCc1cccc(C([O-])(O)[NH2+]C2CCCCC2)c1


[1, 4, 12, 44, 45, 47, 50, 51, 52, 54, 57, 67, 68, 69, 113, 115, 131, 132, 135, 140, 151, 169, 170, 215, 230, 263, 272, 276, 279, 280, 281, 298, 323, 330, 335, 336, 338, 341, 354, 355, 356, 374, 375, 378, 379, 380, 398, 399, 400, 425, 434, 436, 437, 438, 439, 440, 449, 450, 451, 472, 477, 493, 507, 529, 530, 533, 535, 549, 552, 557, 559, 562, 593, 594, 601, 613, 615, 616, 639, 643, 644, 668, 675, 678, 692, 715, 716, 737, 741, 774, 777, 778, 779, 781, 788, 800, 811, 820, 825, 850, 857, 862, 863, 878, 879, 882, 916, 922, 923, 948, 949, 965, 973, 982, 1003, 1004, 1009, 1035, 1067, 1068, 1071, 1072, 1074, 1075, 1076, 1091, 1

In [2]:
print("accuracy: ", (1 - len(wrong_rxn_idx)/len(predicted_product))*100,'%')

accuracy:  90.25618567987738 %


In [18]:
correct_pred_end = 0
num_end = 0
correct_pred = 0
invalid = 0
false_positive = 0

for i in prediction_target_pair:
    check_end = False
    predict_result = i[0]
    ground_truth = i[1]
    if 'END' in ground_truth:
        check_end = True
        ground_truth = i[1].replace('END', '')
        num_end += 1
        if'END' in predict_result:
            predict_result = predict_result.replace('END', '')
            correct_pred_end += 1
    else:
        if 'END' in predict_result:
            predict_result = predict_result.replace('END', '')
            false_positive += 1

    try:
        pred_mol = Chem.MolFromSmiles(predict_result)
        gt_mol = Chem.MolFromSmiles(ground_truth)
        if pred_mol.HasSubstructMatch(gt_mol) and gt_mol.HasSubstructMatch(pred_mol):
            correct_pred += 1
    except:
        invalid += 1
        pass
print(correct_pred, correct_pred_end)
print("Accuracy by substructure match: ", (correct_pred)/len(predicted_product)*100, '%')
print("Accuracy by substructure match with END: ", (correct_pred_end)/num_end*100, '%')
#print('Invalid SMILES: ', invalid/(correct_pred - len(predicted_product))*100, '%')
#print('Invalid SMILES')
print('False positive: ', false_positive)
print('Recall: ', correct_pred_end/(correct_pred_end + false_positive)*100, '%')
print('Precision: ', correct_pred_end/num_end*100, '%')
print('F1 score: ', (2/(pow(correct_pred_end/(correct_pred_end + false_positive), -1) + pow(correct_pred_end/num_end, -1)))*100, '%')

[15:50:03] SMILES Parse Error: extra close parentheses while parsing: CCCC([NH2+]C([O-])(O)C1CC2CN1C(=O)C(C(C)(C)C)NC(=O)Cc1ccccc1)O2)C(O)C(=O)NCC(=O)NC(C(=O)N(C)C)c1ccccc1
[15:50:03] SMILES Parse Error: Failed parsing SMILES 'CCCC([NH2+]C([O-])(O)C1CC2CN1C(=O)C(C(C)(C)C)NC(=O)Cc1ccccc1)O2)C(O)C(=O)NCC(=O)NC(C(=O)N(C)C)c1ccccc1' for input: 'CCCC([NH2+]C([O-])(O)C1CC2CN1C(=O)C(C(C)(C)C)NC(=O)Cc1ccccc1)O2)C(O)C(=O)NCC(=O)NC(C(=O)N(C)C)c1ccccc1'
[15:50:03] SMILES Parse Error: unclosed ring for input: 'CCCC(NC(=O)C1CC2CN1C(=O)C(C(C)(C)C)NC(=O)Cc1ccccc1)C(O)C(=O)NCC(=O)NC(C(=O)N(C)C)c1ccccc1'
[15:50:03] Can't kekulize mol.  Unkekulized atoms: 2 3 6 9 10 11 12 13 14
[15:50:04] SMILES Parse Error: unclosed ring for input: 'Fc1cccc(COc2ccc(N3CCCn4ccc5ncnc4c34)cc2Cl)c1'
[15:50:04] Explicit valence for atom # 18 O, 3, is greater than permitted
[15:50:04] Explicit valence for atom # 12 N, 4, is greater than permitted
[15:50:04] SMILES Parse Error: extra open parentheses for input: 'CCC[NH+](CC(O)

3121 1344
Accuracy by substructure match:  90.91173900378678 %
Accuracy by substructure match with END:  99.26144756277697 %
False positive:  49
Recall:  96.4824120603015 %
Precision:  99.26144756277697 %
F1 score:  97.85220240262105 %


[15:50:04] SMILES Parse Error: unclosed ring for input: 'COC1([O-])NC(C)CN2CCOc3c(Br)sc12'
[15:50:04] SMILES Parse Error: extra close parentheses while parsing: CC1CN2CCOc3c(Br)sc32)C(=O)N1
[15:50:04] SMILES Parse Error: Failed parsing SMILES 'CC1CN2CCOc3c(Br)sc32)C(=O)N1' for input: 'CC1CN2CCOc3c(Br)sc32)C(=O)N1'
[15:50:04] SMILES Parse Error: unclosed ring for input: 'CC(C)[Si](OCc1cc2c(cs1)C1(C)OCCc2ccc(Cl)cc21)(C(C)C)C(C)C'
[15:50:04] SMILES Parse Error: extra open parentheses for input: 'CCOC([O-])(O)CCCCCOc1ccc2c(C#N)c(C#N)c3ccc4cc(O)ccc4c4c(ccc5cc(O)ccc4c3c2c1'


In [62]:
data = []
step = 0
prev_end = 0

wrong_1_ls = []
num_rxn = 0

one_step = 0
two_step = 0
three_step = 0


with open('data/unseen_data/processed/tgt-test.txt', 'r') as target:
    target_lines = target.readlines()
    for line in target_lines:
        if 'END' in line:
            data.append([i for i in range(prev_end, step+1)])
            if step - prev_end == 0:
                wrong_1_ls.append(num_rxn)
                one_step += 1
            elif step - prev_end == 1:
                two_step += 1
            else:
                three_step += 1
            prev_end = step + 1
            num_rxn += 1
        step += 1
        

wrong_1 = 0
wrong_2 = 0
wrong_3 = 0

for i in data:
    for j in i:
        if j in wrong_rxn_idx:
            if len(i) == 1:
                wrong_1 += 1
                print(j)
            elif len(i) == 2:
                wrong_2 += 1
            else:
                wrong_3 += 1
            break


print('One step incorrect: ', wrong_1 - 1)
print('Two step incorrect: ', wrong_2)
print('Three step incorrect: ', wrong_3)
print("accuracy: ", (1 - len(wrong_rxn_idx)/len(predicted_product))*100,'%')
print('One step accuracy: ', (1 - (wrong_1-1)/one_step)*100, '%')
print('One step accuracy, ignoring wrong end token: ', (1 - (wrong_1-3)/one_step)*100, '%')
print('Two step accuracy: ', (1 - wrong_2/two_step)*100, '%')
print('Three step accuracy: ', (1 - wrong_3/three_step)*100, '%')

308
1303
2330
2586
One step incorrect:  3
Two step incorrect:  70
Three step incorrect:  187
accuracy:  88.26099621322459 %
One step accuracy:  78.57142857142857 %
One step accuracy, ignoring wrong end token:  92.85714285714286 %
Two step accuracy:  88.35274542429285 %
Three step accuracy:  74.69553450608932 %
